In [1]:
!pip install openai langchain langchain-openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.1/225.1 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.

In [109]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
SERPAPI = userdata.get('SERPAPI')

In [110]:
from langchain.agents import AgentType, Tool, initialize_agent, load_tools
from langchain_openai import OpenAI

In [111]:
llm = OpenAI(openai_api_key = OPENAI_API_KEY,
             temperature=0)

# Load Tools

In [112]:
tools = load_tools(["serpapi", "llm-math"],
                   llm=llm,
                   serpapi_api_key = SERPAPI)

In [113]:
tools

[Tool(name='Search', description='A search engine. Useful for when you need to answer questions about current events. Input should be a search query.', func=<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0878ac587e4b66f3b18ce94caa93d42603b95c3d10f5843e495e47b7054f9ff8', aiosession=None)>, coroutine=<bound method SerpAPIWrapper.arun of SerpAPIWrapper(search_engine=<class 'serpapi.google_search.GoogleSearch'>, params={'engine': 'google', 'google_domain': 'google.com', 'gl': 'us', 'hl': 'en'}, serpapi_api_key='0878ac587e4b66f3b18ce94caa93d42603b95c3d10f5843e495e47b7054f9ff8', aiosession=None)>),
 Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a 

In [114]:
tools[1]

Tool(name='Calculator', description='Useful for when you need to answer questions about math.', func=<bound method Chain.run of LLMMathChain(llm_chain=LLMChain(prompt=PromptTemplate(input_variables=['question'], template='Translate a math problem into a expression that can be executed using Python\'s numexpr library. Use the output of running this code to answer the question.\n\nQuestion: ${{Question with math problem.}}\n```text\n${{single line mathematical expression that solves the problem}}\n```\n...numexpr.evaluate(text)...\n```output\n${{Output of running the code}}\n```\nAnswer: ${{Answer}}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n```text\n37593 * 67\n```\n...numexpr.evaluate("37593 * 67")...\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 37593^(1/5)\n```text\n37593**(1/5)\n```\n...numexpr.evaluate("37593**(1/5)")...\n```output\n8.222831614237718\n```\nAnswer: 8.222831614237718\n\nQuestion: {question}\n'), llm=OpenAI(client=<openai.resources.completions.Completions ob

In [115]:
tools[1].name , tools[1].description

('Calculator', 'Useful for when you need to answer questions about math.')

# Initialize the agent

In [116]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

In [117]:
print(agent.agent.llm_chain.prompt.template)

Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [119]:
agent.run("What is the weather in Elkart, Indiana?")



> Entering new AgentExecutor chain...
 I should search for the current weather in Elkart, Indiana.
Action: Search
Action Input: "weather in Elkart, Indiana"
Observation: {'type': 'weather_result', 'temperature': '31', 'unit': 'Fahrenheit', 'precipitation': '0%', 'humidity': '92%', 'wind': '7 mph', 'location': 'Elkhart, IN', 'date': 'Friday 8:00 AM', 'weather': 'Mostly cloudy'}
Thought: I now know the current weather in Elkart, Indiana.
Final Answer: The current weather in Elkart, Indiana is mostly cloudy with a temperature of 31 degrees Fahrenheit, 0% chance of precipitation, 92% humidity, and 7 mph winds.

> Finished chain.


'The current weather in Elkart, Indiana is mostly cloudy with a temperature of 31 degrees Fahrenheit, 0% chance of precipitation, 92% humidity, and 7 mph winds.'

In [120]:
agent.run("What time is it right now?")



> Entering new AgentExecutor chain...
 You should always think about what to do
Action: Search
Action Input: "current time"
Observation: 8:53 AM
Thought: I now know the final answer
Final Answer: 8:53 AM

> Finished chain.


'8:53 AM'